In [7]:
%run ../Data-Preprocessing.ipynb
%run ../U-Net.ipynb
%run ../Train-Eval-Utils.ipynb

import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [7]:
convert_ct_dataset_to_slices('Task06_Lung', 'Lung_Train', 'Lung_Val', 'Lung_Test', val_split=0.1, test_split=0.1, negative_downsampling_rate=5)

FileExistsError: [Errno 17] File exists: 'Lung_Train/images'

In [8]:
transform = transforms.Compose([
    transforms.Resize((128, 128), antialias=False),
    transforms.ConvertImageDtype(torch.float)
])

train_images = os.path.join('Lung_Train', 'images')
train_labels = os.path.join('Lung_Train', 'labels')

train_dataset = CTDataset(train_images, train_labels, image_transform=transform, label_transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

val_images = os.path.join('Lung_Val', 'images')
val_labels = os.path.join('Lung_Val', 'labels')

val_dataset = CTDataset(val_images, val_labels, image_transform=transform, label_transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

test_images = os.path.join('Lung_Test', 'images')
test_labels = os.path.join('Lung_Test', 'labels')

test_dataset = CTDataset(test_images, test_labels, image_transform=transform, label_transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

In [3]:
model = UNet()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
best_model, best_loss = train_loop_with_validation(model, 50, train_loader, val_loader)
print(f'Best loss achieved on the validation set: {best_loss}')

Epoch 1/50 started...


KeyboardInterrupt: 